In [ ]:
system_prompt = 'You are a programmer,you use python,sympy to solve problems,set variables,exprestions,set Interval to give range of variable,and use solve() function to calculation.only give a python code'

### load data

In [1]:
import json
from datasets import load_dataset, Dataset
with open('conic10k/test.json', 'r', encoding='utf-8') as file:
    raw_conic_tr = json.load(file)
datasett = load_dataset('json', data_files='conic10k/test.json', split=None)
# dataset1['train']['text']
# len(dataset1)
datasett['train'][0]['text']

'如果双曲线的两个焦点分别为$F_{1}(-3,0)$, $F_{2}(3,0)$ ,一条渐近线方程为$y=\\sqrt{2} x$，则该双曲线的方程为?'

### prompt to have answer

In [2]:
#init data

ans = {'Ai_answer':[],'real_answer':[],'primary_output':[]}
flag = 1000
my_dataset = datasett
print(my_dataset)

DatasetDict({
    train: Dataset({
        features: ['query_spans', 'text', 'query_expressions', 'process', 'fact_spans', 'answer_expressions', 'fact_expressions'],
        num_rows: 2069
    })
})


In [3]:
#将大模型输出的结果中的代码提取并运行，将结果返回
import re
import sys
from io import StringIO
def to_py_and_run(text):
    pattern = r'```python\n(.*?)\n```'
    match = re.search(pattern, text, re.DOTALL)
    run_ans = 'None'
    if match:
        code_block = match.group(1).strip()  # 提取代码块并去除首尾空白
        
        # 截取最后一行提取代码结果
        last_line = code_block.rstrip().rsplit('\n', 1)[-1]

        # #最后一行可能是print(answer_expr),也可能是answer_expr，我们归一化处理
        ppattern = r"print\((.*)\)" 
        pmatch = re.search(ppattern, last_line)
        pmatch
        if pmatch:
            no_use=1 # jump
        else:
            code_block = code_block + '\n\nprint('+ last_line+')'
        
        # with open('outputcode.py', 'w') as file:
        #     file.write(code_block)  # 将代码写入文件
        # #为了保存输出，使用重定向


        old_stdout = sys.stdout
        new_stdout = StringIO()
        sys.stdout = new_stdout

        # 运行生成的Python文件 保存输出到返回值，并恢复输出定向
        try:
            exec(code_block,globals())
            output = new_stdout.getvalue()

        except:
            run_ans = 'code_error'
        else:
            # print('printsuccess')
            run_ans = output.strip('\n')
        sys.stdout = old_stdout
            
    return run_ans

In [16]:
from sympy import symbols, Eq, solve
# Define symbols
p = symbols('p', positive=True)
# Given |FQ|
FQ = 6
# For the parabola y^2 = 2px, focus F is at (p/2, 0)
# Since PF is perpendicular to the x-axis, P has coordinates (p/2, sqrt(2p*(p/2))) = (p/2, p)
# PQ is perpendicular to OP, so Q is on the x-axis and has coordinates (p, 0)
# The distance from F to Q is given as |FQ| = 6
# Thus, |FQ| = |p - p/2| = |p/2| = 6, which gives us p = 12
# Calculate p
p_value = solve(Eq(p / 2, FQ), p)[0]
# The directrix or latus rectum of the parabola is at x = -p/2
directrix = -p_value / 2
# Equation of the directrix
print(directrix)

-6


In [10]:
from gradio_client import Client
client = Client("Qwen/Qwen2-72B-Instruct")
i=flag
while i < len(my_dataset['train']['text']) and i<1500:
  mathproblem = my_dataset['train'][i]['text']
  problemans = my_dataset['train'][i]['answer_expressions']
  result = client.predict(
  query=mathproblem,
  history=[],
  system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
  api_name="/model_chat"
  )
  print('finish predict')
  ans['primary_output'].append(result[1][0][1])
  code_result = to_py_and_run(result[1][0][1])
  if code_result =='code_error':
    stop_point = 0
    stop_limit = 1
    while code_result =='code_error' and stop_point < stop_limit:
      stop_point = stop_point+1
      print('error time:',stop_point)
      new_result = client.predict(
        query='you gave me a code to solve a problem,code:'+code_result+'problem:'+mathproblem+'pleas debug the code,then only give me a corrected python code',
        history=[],
        system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
        api_name="/model_chat"
      )
      code_result = to_py_and_run(new_result[1][0][1])

  ans['Ai_answer'].append(code_result)
  ans['real_answer'].append(problemans)
  print(i)
  print(mathproblem)
  print(code_result)
  print(problemans)
  with open('test_predict_3.json', 'w', encoding='utf-8') as file:
    json.dump(ans, file)
  i = i+1
  flag = i

Loaded as API: https://qwen-qwen2-72b-instruct.hf.space ✔
finish predict
1356
已知双曲线$\frac{x^{2}}{m}-y^{2}=1$和椭圆$\frac{x^{2}}{12}+\frac{y^{2}}{4}=1$焦点相同，则该双曲线的方程为?
x**2/7 - y**2 == 1
x^2/7-y^2=1
finish predict
1357
双曲线$\frac{x^{2}}{36}-\frac{y^{2}}{m}=1$的焦距为$18$，则双曲线的渐近线方程为?
-1.11803398874989*x + y 1.11803398874989*x + y

finish predict
1358
双曲线$3 my^{2}-mx^{2}=3$的一个焦点是$(0 , 2)$，则$m$的值是?
[-1.99999999999999]
1
finish predict
1359
设双曲线$\frac{x^{2}}{a^{2}}-\frac{y^{2}}{b^{2}}=1(a>0, b>0)$的右焦点是$F$，左、右顶点分别是$A_{1}$、$A_{2}$，过$F$做$x$轴的垂线交双曲线于$B$、$C$两点，若$A_{1} B \perp A_{2} C$，则双曲线的离心率为?
sqrt(2)
sqrt(2)
finish predict
error time: 1
1360
抛物线$y^{2}=2 p x(p>0)$上一点$M(1, m)(m>0)$到其焦点的距离为$5$, 双曲线$\frac{x^{2}}{a}-y^{2}=1$的左顶点为$A$. 若双曲线的一条渐近线与直线$AM$平行. 则实数$a$等于?
[1/9]
1/9
finish predict
1361
已知椭圆与$x$轴相切，两个焦点坐标为$F_{1}(1,1)$, $F_{2}(5,2)$，则其长轴长为?
sqrt(21)

finish predict
1362
若椭圆$\frac{x^{2}}{k+4}+\frac{y^{2}}{9}=1$的一个焦点是$(0,2)$，则实数$k$=?
1
1
finish predict
error time: 1
1363
已知双曲线$\frac{x^{2}}{a^{2}}-\fra

In [5]:
# query flag
flag

1050

In [11]:
import pandas as pd
# 需要库：openpyxl
# 将字典转换为pandas DataFrame
memory_df = pd.DataFrame(ans)
 
# 将DataFrame写入Excel文件
# !!!!记得修改你的文件名
memory_df.to_excel('qwen2_codeversion_3.xlsx', index=1)